In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

from ai.TrashClassifierAE import TrashClassifier
from ai.DataLoader2 import DataLoader
from env import *

EPOCHS = 200
ETA = 1e-2

In [2]:
model = TrashClassifier(fine_tune=False)
model = model.cuda()

FeatureAE was loaded.


In [3]:
train_dloader = DataLoader(TRAIN_DATA_PATH, TRASH_CAT)
valid_dloader = DataLoader(VALID_DATA_PATH, TRASH_CAT)

Number of data: 1241
Number of batch: 155
Number of data: 112
Number of batch: 14


In [4]:
optimizer = optim.Adam(model.parameters(), lr=ETA)
criterion = nn.NLLLoss()

In [5]:

top_valid_acc = 0.0

for e in range(EPOCHS):
    
    train_loss = 0.0
    train_acc = 0.0
    valid_loss = 0.0
    valid_acc = 0.0
    
    for x_batch, y_batch in train_dloader.next_batch():
        x_batch = torch.FloatTensor(x_batch).cuda()
        y_batch = torch.LongTensor(y_batch).cuda()
        
        logps = model(x_batch)
        loss = criterion(logps, y_batch)
        
        with torch.no_grad():
            train_loss += loss.item()
            ps = torch.exp(logps)
            
            ps_k, cls_k = ps.topk(1, dim=1)
            equal = cls_k == y_batch.view(*cls_k.size())
            train_acc += torch.mean(equal.type(torch.FloatTensor))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (e+1) % 10 == 0:
        with torch.no_grad():
            model.eval()

            for x_batch, y_batch in valid_dloader.next_batch():
                x_batch = torch.FloatTensor(x_batch).cuda()
                y_batch = torch.LongTensor(y_batch).cuda()

                logps = model(x_batch)
                loss = criterion(logps, y_batch)

                valid_loss += loss.item()

                ps = torch.exp(logps)
                ps_k, cls_k = ps.topk(1, dim=1)
                equal = cls_k == y_batch.view(*cls_k.size())
                valid_acc += torch.mean(equal.type(torch.FloatTensor))

            train_loss /= len(train_dloader)
            train_acc /= len(train_dloader)
            valid_loss /= len(valid_dloader)
            valid_acc /= len(valid_dloader)

            print(f"Epochs: {e+1}/{EPOCHS}")
            print(f"Train loss: {train_loss:.8f}")
            print(f"Train acc: {train_acc:.8f}")
            print(f"Valid loss: {valid_loss:.8f}")
            print(f"Valid acc: {valid_acc:.8f}")
            
            if valid_acc > top_valid_acc:
                model.save(CLF_CKPT_PATH)
                top_valid_acc = valid_acc

            model.train()

Epochs: 10/200
Train loss: 0.22257056
Train acc: 0.92473114
Valid loss: 0.86017824
Valid acc: 0.83333337
Trash Classifier was saved.
Epochs: 20/200
Train loss: 0.23602918
Train acc: 0.93010753
Valid loss: 0.80343890
Valid acc: 0.85714287
Trash Classifier was saved.
Epochs: 30/200
Train loss: 0.14619873
Train acc: 0.96451622
Valid loss: 0.70020685
Valid acc: 0.83333337
Epochs: 40/200
Train loss: 0.13105793
Train acc: 0.96344084
Valid loss: 1.42274215
Valid acc: 0.73809516
Epochs: 50/200
Train loss: 0.11169962
Train acc: 0.97096783
Valid loss: 0.55580070
Valid acc: 0.88095242
Trash Classifier was saved.
Epochs: 60/200
Train loss: 0.10642085
Train acc: 0.97526896
Valid loss: 0.33582355
Valid acc: 0.91666669
Trash Classifier was saved.
Epochs: 70/200
Train loss: 0.09485366
Train acc: 0.97526884
Valid loss: 0.84806870
Valid acc: 0.88095242
Epochs: 80/200
Train loss: 0.12986720
Train acc: 0.97096795
Valid loss: 0.66600785
Valid acc: 0.85714281
Epochs: 90/200
Train loss: 0.12259644
Train acc:

In [6]:
1+2

3